In [13]:
import pandas as pd
import gzip
import json
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances,cosine_similarity
from sklearn.metrics import mean_squared_error
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

In [14]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Beauty_5.json.gz')
df2 = getDF('meta_Beauty.json.gz')

In [15]:
df = df.drop(["reviewerName", "summary", "reviewTime"], axis=1, inplace=False)
df2 = df2.drop(["imUrl", "related", "brand"], axis=1, inplace = False)
new_df = df.merge(df2, how='inner', on='asin')
new_df['helpful_num'] = df['helpful'].apply(lambda x: x[0])
new_df['helpful_denom'] = df['helpful'].apply(lambda x: x[1])
new_df1 = new_df.drop("helpful", axis = 1)
new_df1.head()
count = new_df1.groupby("asin", as_index=False).count()

new_df2 = pd.merge(new_df1, count, how='right', on=['asin'])
new_df2.columns
new_df2["numberReview"] = new_df2["reviewerID_y"]
#Filter out products with review numbers >= 50
dfCount = new_df2[new_df2.numberReview >= 50]
dfCount

,reviewerID_x,asin,reviewText_x,overall_x,unixReviewTime_x,description_x,title_x,salesRank_x,categories_x,price_x,...,overall_y,unixReviewTime_y,description_y,title_y,salesRank_y,categories_y,price_y,helpful_num_y,helpful_denom_y,numberReview
105,AEEJGDRRGLBB1,B000050B6U,"I have long, thick, and straight hair. I bough...",2.0,1376438400,"Welcome to the world of advanced styling, wher...","Instant Heat Curling Iron, 1-Inch",{'Beauty': 3225},"[[Beauty, Hair Care, Styling Tools, Irons, Cur...",13.95,...,51,51,51,51,51,51,51,51,51,51
106,A2K6LI89NAD2J2,B000050B6U,I love that it has more heat settings than a l...,5.0,1365638400,"Welcome to the world of advanced styling, wher...","Instant Heat Curling Iron, 1-Inch",{'Beauty': 3225},"[[Beauty, Hair Care, Styling Tools, Irons, Cur...",13.95,...,51,51,51,51,51,51,51,51,51,51
107,A1P9UMP1XSE6MI,B000050B6U,I don't curl my hair every day so I didn't nee...,5.0,1391385600,"Welcome to the world of advanced styling, wher...","Instant Heat Curling Iron, 1-Inch",{'Beauty': 3225},"[[Beauty, Hair Care, Styling Tools, Irons, Cur...",13.95,...,51,51,51,51,51,51,51,51,51,51
108,A4WX0FZ1MMAUH,B000050B6U,I bought the biggest size of this curling just...,5.0,1395446400,"Welcome to the world of advanced styling, wher...","Instant Heat Curling Iron, 1-Inch",{'Beauty': 3225},"[[Beauty, Hair Care, Styling Tools, Irons, Cur...",13.95,...,51,51,51,51,51,51,51,51,51,51
109,AHP55PFHJL0H8,B000050B6U,It gets too hot and is hard to handle without ...,2.0,1391558400,"Welcome to the world of advanced styling, wher...","Instant Heat Curling Iron, 1-Inch",{'Beauty': 3225},"[[Beauty, Hair Care, Styling Tools, Irons, Cur...",13.95,...,51,51,51,51,51,51,51,51,51,51
110,ASPABROV8R7M5,B000050B6U,I like the size of the barrel on this iron and...,1.0,1380153600,"Welcome to the world of advanced styling, wher...","Instant Heat Curling Iron, 1-Inch",{'Beauty': 3225},"[[Beauty, Hair Care, Styling Tools, Irons, Cur...",13.95,...,51,51,51,51,51,51,51,51,51,51
111,A3T4OHEAHOJ700,B000050B6U,I didn't expect too much when I purchased this...,5.0,1395705600,"Welcome to the world of advanced styling, wher...","Instant Heat Curling Iron, 1-Inch",{'Beauty': 3225},"[[Beauty, Hair Care, Styling Tools, Irons, Cur...",13.95,...,51,51,51,51,51,51,51,51,51,51
112,AWR80FA3IQ633,B000050B6U,The 1/2&#34; barrel is perfect for styling sho...,5.0,1378684800,"Welcome to the world of advanced styling, wher...","Instant Heat Curling Iron, 1-Inch",{'Beauty': 3225},"[[Beauty, Hair Care, Styling Tools, Irons, Cur...",13.95,...,51,51,51,51,51,51,51,51,51,51
113,A3TQZLWYCQXLCQ,B000050B6U,This came quick and packaged just like I bough...,5.0,1355529600,"Welcome to the world of advanced styling, wher...","Instant Heat Curling Iron, 1-Inch",{'Beauty': 3225},"[[Beauty, Hair Care, Styling Tools, Irons, Cur...",13.95,...,51,51,51,51,51,51,51,51,51,51
114,A2RUV6V9CWZAA0,B000050B6U,,3.0,1300838400,"Welcome to the world of advanced styling, wher...","Instant Heat Curling Iron, 1-Inch",{'Beauty': 3225},"[[Beauty, Hair Care, Styling Tools, Irons, Cur...",13.95,...,51,51,51,51,51,51,51,51,51,51


In [16]:
content_df = pd.read_csv('content_based_sim.csv')
content_df = content_df.set_index('asin')

In [17]:
content_df.head()

,salesRank,price,categories2,categories1,activ ingredi,affect natur,affect natur smell,affect natur smell skin,affect time,affect time fragranc,...,xa instanc mood stress,xa similarli,xa similarli factor,xa similarli factor dri,xa skin,xa skin affect,xa skin affect time,xa xa,xa xa similarli,xa xa similarli factor
asin,,,,,,,,,,,,,,,,,,,,,
0205616461,461765,24.878165,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0558925278,402875,24.878165,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0733001998,540255,24.878165,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0737104473,931125,24.878165,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0762451459,0,19.980000,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
len(content_df.index.unique())

259204

In [11]:
bins = [0, 10, 20, 30, 40, 50, 250]
labels = [1,2,3,4,5,6]
content_df['binned_price'] = pd.cut(content_df['price'], bins=bins, labels=labels)

In [12]:
content_df.head()

,salesRank,price,categories2,categories1,activ ingredi,affect natur,affect natur smell,affect natur smell skin,affect time,affect time fragranc,...,xa similarli,xa similarli factor,xa similarli factor dri,xa skin,xa skin affect,xa skin affect time,xa xa,xa xa similarli,xa xa similarli factor,binned_price
asin,,,,,,,,,,,,,,,,,,,,,
0205616461,461765,24.878165,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
0558925278,402875,24.878165,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
0733001998,540255,24.878165,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
0737104473,931125,24.878165,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
0762451459,0,19.980000,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [7]:
content_array = content_df.values

In [9]:
content_array.dump("content_array.dat")

In [10]:
content_array = np.load("content_array.dat")

In [5]:
np.savetxt("content.csv", content_array, delimiter = ',')

In [6]:
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
content_sim = linear_kernel(content_array, content_array)

In [ ]:
content_sim.dump("content_sim.dat")